In [1]:
# auto reload
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from mtt.simulator import position_image
from mtt.utils import compute_ospa
from tqdm.notebook import tqdm


In [3]:
filter_name = "glmb"
cols = ["scale", "simulation_idx", "step_idx", "posx_m", "posy_m", "exist_prob"]

df_truth = pd.read_csv(
    f"../data/sim_data/{filter_name}/{filter_name}_truth.csv", low_memory=False
)
df_filter = pd.read_csv(
    f"../data/sim_data/{filter_name}/{filter_name}_estimates.csv", low_memory=False
)
df_combined = (
    pd.concat([df_truth, df_filter], axis=0, keys=["truth", "filter"])
    .reset_index(level=0, names=["truth"])
    .rename(columns={"time": "step_idx"})
)
integer_columns = ["scale", "step_idx"]
df_combined[integer_columns] = df_combined[integer_columns].astype(int)


In [4]:
n_scales = df_combined["scale"].unique().size
n_simulations = df_combined["simulation_idx"].max() + 1
n_steps = df_combined["step_idx"].max() + 1


In [5]:
def to_image(df: pd.DataFrame, scale: int):
    # LMCO does not center their sims around 0,0
    # 4000, 5000, 7000
    if scale == 1:
        center = 2000.0
    elif scale == 2:
        center = 2500.0
    elif scale == 3:
        center = 3500.0
    else:
        raise ValueError(f"Unknown scale {scale}")

    na_positions = df[["posx_m", "posy_m"]].isna().any(axis=1)
    positions = df[["posx_m", "posy_m"]].to_numpy()[~na_positions] - center
    weights = df["exist_prob"].fillna(1.0).to_numpy()[~na_positions]
    assert len(positions) == len(weights)
    return position_image(
        window_width=1000 * scale,
        size=128 * scale,
        sigma=10.0,
        target_positions=positions,
        device="cuda",
        weights=weights,
    )


results = []
groupby_columns = ["simulation_idx", "step_idx", "scale"]
gb = df_combined.groupby(groupby_columns, as_index=False, sort=True)
for (idx, df) in tqdm(gb, total=len(gb)):
    simulation_idx, step_idx, scale = idx
    truth = df[df["truth"] == "truth"]
    filter = df[df["truth"] == "filter"]

    truth_positions = truth[["posx_m", "posy_m"]].dropna().to_numpy()
    filter_positions = (
        filter.query("exist_prob > 0.5")[["posx_m", "posy_m"]].dropna().to_numpy()
    )
    ospa = compute_ospa(truth_positions, filter_positions, 500, 2)

    truth_image = to_image(truth, scale)
    filter_image = to_image(filter, scale)
    mse = (truth_image - filter_image).pow(2).mean().item()
    results.append(
        dict(
            scale=scale,
            simulation_idx=simulation_idx,
            step_idx=step_idx,
            ospa=ospa,
            mse=mse,
            filter=filter_name,
            cardinality_truth=len(truth_positions),
            cardinality_estimate=filter["exist_prob"].sum(),
        )
    )
df_results = pd.DataFrame(results)


  0%|          | 0/30000 [00:00<?, ?it/s]

In [ ]:
df_results.to_csv(f"../data/out/{filter_name}_summary.csv", index=False)
